# II. Min hashing

In [1]:
import pandas as pd
import numpy as np
import collections
import itertools
import random
from tqdm import tqdm
from math import floor, sqrt
import ast

Cargar datos preprocesados

In [4]:
resumen = pd.read_csv('preprocesado.csv', converters={"shingles": ast.literal_eval})

Cargar vocabulario de shingles previamente computado y mostrar su largo

In [5]:
with open('shingles_sample.txt','r') as f:
   shingles_sample = ast.literal_eval(f.read())
un = shingles_sample
len(un)

9958

## Crear funciones de hash

Primero hallamos un número primo más grande que el universo de shingles

In [7]:

def is_prime(n):
  for i in range(2,int(sqrt(n))+1):
    if (n%i) == 0:
      return False
  return True

numb = len(un)+1
while True:
    result = is_prime(numb)
    if result:
       break
    numb+=1
print("Primo encontrado: ", numb)

Primo encontrado:  9967


Creamos 20 funciones de hash, usando un método tomado de material de clases

In [8]:
def crear_hash(a, b, p, n):
    def f(x):
        return ((a * x + b) % p) % n
    return f
h = []
n = len(un)
p = numb
num_hash = 20
for i in range(num_hash):
    a = random.randint(1,p-1)
    b = random.randint(1,p-1)
    h.append(crear_hash(a,b,p,n))

## Firma de Hash

Calcularemos la firma de hash dividiendo el resumen en 10 chunks.

In [2]:
sorted_union = sorted(un)
def chunk_signature(chunk):
    n = len(chunk)
    FH = np.full((num_hash, n), n+100)
    ### Iterar por elementos del vocabulario
    for e in tqdm(range(0,len(sorted_union))): # |union_shingles| = 9967
       
        ### iteramos cada columna, correspondiente a la cantidad de textos (tweets en nuestro caso)  
        for i in range(0,n): # |cols| = 208.139

            ### si el elemento e no está en el texto i, no hacemos nada
            if sorted_union[e] not in chunk.shingles[i]:
                pass
            
            ### si el elemento si está, computamos los valores de h(e) y 
            else:
                for j in range(0,num_hash): # j es función de hash |hash| = 20, filas de la matriz
                    FH[j][i] = min(FH[j][i],h[j](e))
    return FH

NameError: name 'un' is not defined

Calcular la firma de hash de cada chunk e ir escribiendo a disco

In [11]:
n_of_chunks = 10
chunk_size = floor(len(resumen)/n_of_chunks)
chunk = resumen.iloc[:chunk_size,:]
FH = chunk_signature(chunk)
np.savetxt("fhs/file0.csv", FH, delimiter=",")
del FH
for i in tqdm(range(1, n_of_chunks)):
    if i != (n_of_chunks-1):
        chunk = resumen.iloc[chunk_size*i:chunk_size*(i+1),:]
    else:
        chunk = resumen.iloc[chunk_size*i:,:]
    chunk = chunk.reset_index()
    FH = chunk_signature(chunk)
    np.savetxt(f"fhs/file{i}.csv", FH, delimiter=",")
    del FH